In [1]:
!pip install --quiet geopandas rasterio shapely scipy matplotlib pyproj tqdm mapclassify rtree rio-cogeo rasterstats

import os, json, math, logging, datetime
import numpy as np
import geopandas as gpd
import rasterio
from rasterio.transform import from_origin
from rasterio.features import rasterize, shapes as rio_shapes
from rasterio.windows import Window
from rasterio.enums import Resampling
from rasterio.warp import reproject, Resampling as WarpResampling
try:
    from rasterio.enums import MergeAlg
    _HAS_MERGEALG = True
except Exception:
    MergeAlg = None
    _HAS_MERGEALG = False

from shapely.geometry import shape as shapely_shape, Point
from scipy.ndimage import gaussian_filter
from tqdm import tqdm
import matplotlib.pyplot as plt
from google.colab import files

logging.basicConfig(level=logging.INFO, format='%(asctime)s %(levelname)s %(message)s')
log = logging.getLogger("lighting_pipeline")

# upload lamp gpkg
print("upload file")
uploaded = files.upload()
if len(uploaded) == 0:
    raise SystemExit("No file uploaded")
VECTOR_PATH = list(uploaded.keys())[0]
log.info(f"Uploaded: {VECTOR_PATH}")

# output files
OUTPUT_RASTER = "lighting_model_highres_disk.tif"
OUTPUT_COG = "lighting_model_highres_disk_cog.tif"
VECTOR_GRID = "lighting_vector_grid_nonzero.gpkg"
HIST_IMAGE = "lighting_histogram.png"
HIST_LOG_IMAGE = "lighting_histogram_log.png"
HIST_CSV = "lighting_histogram.csv"
METADATA_JSON = "lighting_metadata.json"

PIXEL_SIZE = 5
GAUSSIAN_SIGMA_PIX = 10
CHUNK_SIZE = 2000            # tile size in pixels
GRID_CELL_SIZE = 50
NORMALIZE_TOTAL = True       # preserve total lamp count
NODATA_VALUE = -9999.0
NBINS_HIST = 30
SEED = 42

np.random.seed(SEED)

# load and validate lamp gpkg
lamps = gpd.read_file(VECTOR_PATH)
log.info(f"Loaded lamp points: {len(lamps)}; original CRS: {lamps.crs}")

TARGET_CRS = "EPSG:25832"
if lamps.crs is None or lamps.crs.to_string() != TARGET_CRS:
    log.info(f"Reprojecting to {TARGET_CRS}")
    lamps = lamps.to_crs(TARGET_CRS)
log.info(f"Active CRS: {lamps.crs}")

if len(lamps) == 0:
    raise SystemExit("ERROR: No lamp features found in input file.")

# check spatial index exists
try:
    _ = lamps.sindex
    spatial_index = lamps.sindex
except Exception:
    spatial_index = None
    log.warning("Spatial index unavailable; spatial queries slower.")

# raster grid metadata
minx, miny, maxx, maxy = lamps.total_bounds
pad_m = PIXEL_SIZE
minx -= pad_m; miny -= pad_m; maxx += pad_m; maxy += pad_m

width = int(math.ceil((maxx - minx)/PIXEL_SIZE))
height = int(math.ceil((maxy - miny)/PIXEL_SIZE))
transform = from_origin(minx, maxy, PIXEL_SIZE, PIXEL_SIZE)
log.info(f"Raster: width={width}, height={height}, transform={transform}")

if width <= 0 or height <= 0:
    raise SystemExit("Invalid raster dimensions.")

def rasterize_additive(shapes_iter, out_shape, transform_local, dtype='uint32'):
# Rasterize features additively
    if _HAS_MERGEALG and MergeAlg is not None:
        try:
            return rasterize(
                shapes=shapes_iter,
                out_shape=out_shape,
                transform=transform_local,
                fill=0,
                dtype=dtype,
                merge_alg=MergeAlg.add
            )
        except TypeError:
            pass
    # fallback: loop per-shape
    out = np.zeros(out_shape, dtype=dtype)
    for geom, val in shapes_iter:
        single = rasterize([(geom,val)], out_shape=out_shape, transform=transform_local, fill=0, dtype=dtype)
        out += single
    return out

# chunked rasterization & gaussian smoothing
log.info("Starting chunked additive rasterization + Gaussian smoothing...")
halo = int(math.ceil(3*GAUSSIAN_SIGMA_PIX))
halo = min(halo, max(0, CHUNK_SIZE//2 - 1))
log.info(f"Halo (pixels): {halo}")

meta = {
    'driver':'GTiff','height':height,'width':width,'count':1,
    'dtype':'float32','crs':TARGET_CRS,'transform':transform,
    'tiled':True,'blockxsize':CHUNK_SIZE,'blockysize':CHUNK_SIZE,
    'compress':'lzw','BIGTIFF':'YES','nodata':NODATA_VALUE
}

with rasterio.open(OUTPUT_RASTER,'w',**meta) as dst:
    burn_total_est = 0
    for row_start in tqdm(range(0,height,CHUNK_SIZE), desc="rows"):
        for col_start in range(0,width,CHUNK_SIZE):
            r0 = max(0,row_start-halo); c0=max(0,col_start-halo)
            r1 = min(height,row_start+CHUNK_SIZE+halo)
            c1 = min(width,col_start+CHUNK_SIZE+halo)
            win_h = r1-r0; win_w = c1-c0
            x0 = minx + c0*PIXEL_SIZE; x1 = minx + c1*PIXEL_SIZE
            y1 = maxy - r0*PIXEL_SIZE; y0 = maxy - r1*PIXEL_SIZE

            try:
                idx = list(spatial_index.intersection((x0,y0,x1,y1)))
            except Exception:
                idx = list(lamps.index)
            lamps_tile = lamps.iloc[idx]

            count_arr = np.zeros((win_h,win_w),dtype='uint32')
            if len(lamps_tile)>0:
                shapes_iter = ((geom,1) for geom in lamps_tile.geometry)
                burned = rasterize_additive(shapes_iter,out_shape=(win_h,win_w),
                                            transform_local=from_origin(x0,y1,PIXEL_SIZE,PIXEL_SIZE),
                                            dtype='uint32')
                count_arr += burned

            # Smooth
            count_f = count_arr.astype('float32')
            smoothed = gaussian_filter(count_f,sigma=GAUSSIAN_SIGMA_PIX,mode='constant',cval=0.0) if np.any(count_f) else count_f

            # Write center chunk (exclude halo)
            r_c0 = row_start - r0; c_c0 = col_start - c0
            r_c1 = r_c0 + min(CHUNK_SIZE,height-row_start)
            c_c1 = c_c0 + min(CHUNK_SIZE,width-col_start)
            chunk_center = smoothed[r_c0:r_c1, c_c0:c_c1]

            write_win = Window(col_start,row_start,chunk_center.shape[1],chunk_center.shape[0])
            dst.write(chunk_center.astype('float32'),1,window=write_win)
            burn_total_est += int(count_arr[r_c0:r_c1,c_c0:c_c1].sum())

    dst.update_tags(lamp_count=len(lamps),pixel_size=PIXEL_SIZE,bounds=f"{minx},{miny},{maxx},{maxy}")
log.info(f"Rasterization + smoothing done -> {OUTPUT_RASTER}")

# build raster overviews
with rasterio.open(OUTPUT_RASTER,'r+') as src:
    factors = [2,4,8,16]; src.build_overviews(factors,Resampling.average)
    src.update_tags(ns='rio_overview',resampling='average')
log.info("Overviews built")

# stats & histogram
def welford_update(count,mean,M2,data_array):
    data = data_array.ravel()
    mask = (data != NODATA_VALUE) & np.isfinite(data)
    if not np.any(mask):
        return count,mean,M2
    x = data[mask].astype('float64')
    for val in x:
        count +=1; delta=val-mean; mean+=delta/count; delta2=val-mean; M2+=delta*delta2
    return count,mean,M2

count=0; mean=0.0; M2=0.0; min_val=np.inf; max_val=-np.inf; nonzero_pixels=0; total_pixels=0
with rasterio.open(OUTPUT_RASTER) as src:
    for _,window in src.block_windows(1):
        data=src.read(1,window=window,boundless=True,fill_value=NODATA_VALUE)
        total_pixels+=data.size
        mask=(data!=NODATA_VALUE)&np.isfinite(data)
        if np.any(mask):
            values=data[mask]; nonzero_pixels+=int(np.count_nonzero(values>0))
            min_val=min(min_val,float(np.nanmin(values))); max_val=max(max_val,float(np.nanmax(values)))
            count,mean,M2=welford_update(count,mean,M2,data)
variance=(M2/(count-1)) if count>1 else 0.0
std=math.sqrt(variance) if variance>=0 else float('nan')
stats={"pixels_total":int(total_pixels),"pixels_with_data":int(count),"nonzero_pixels":int(nonzero_pixels),
       "min":float(min_val) if min_val!=np.inf else float('nan'),
       "max":float(max_val) if max_val!=-np.inf else float('nan'),
       "mean":float(mean),"std":float(std)}
log.info("Raster stats computed (streaming):")
for k,v in stats.items(): log.info(f"  {k}: {v}")

bins=np.linspace(stats['min'],stats['max'],NBINS_HIST+1) if np.isfinite(stats['min']) and np.isfinite(stats['max']) and stats['min']<stats['max'] else np.linspace(0,1,NBINS_HIST+1)
hist_counts=np.zeros(len(bins)-1,dtype=np.int64)
with rasterio.open(OUTPUT_RASTER) as src:
    for _,window in src.block_windows(1):
        data=src.read(1,window=window,boundless=True,fill_value=NODATA_VALUE)
        mask=(data!=NODATA_VALUE)&np.isfinite(data)
        if np.any(mask):
            h,_=np.histogram(data[mask],bins=bins)
            hist_counts+=h

np.savetxt(HIST_CSV,np.c_[bins[:-1],bins[1:],hist_counts],delimiter=",",
           header="bin_start,bin_end,count",comments="")
centers=(bins[:-1]+bins[1:])/2.0; widths=(bins[1]-bins[0])
plt.figure(figsize=(8,4)); plt.bar(centers,hist_counts,width=widths,edgecolor='black')
plt.title("Lighting Intensity Histogram"); plt.xlabel("Intensity"); plt.ylabel("Frequency")
plt.tight_layout(); plt.savefig(HIST_IMAGE,dpi=150); plt.close()
plt.figure(figsize=(8,4)); plt.bar(centers,hist_counts,width=widths,edgecolor='black'); plt.yscale('log')
plt.title("Lighting Intensity Histogram (log)"); plt.xlabel("Intensity"); plt.ylabel("Frequency (log)")
plt.tight_layout(); plt.savefig(HIST_LOG_IMAGE,dpi=150); plt.close()
log.info(f"Histogram saved: {HIST_CSV}, {HIST_IMAGE}, {HIST_LOG_IMAGE}")

# normalization
if NORMALIZE_TOTAL:
    lamp_count = len(lamps)
    current_sum = 0.0
    with rasterio.open(OUTPUT_RASTER) as src:
        for _,window in src.block_windows(1):
            data=src.read(1,window=window,boundless=True,fill_value=NODATA_VALUE)
            mask=(data!=NODATA_VALUE)&np.isfinite(data)
            if np.any(mask): current_sum+=float(np.sum(data[mask]))
    if current_sum>0:
        scale=float(lamp_count)/current_sum
        log.info(f"Normalizing raster by scale factor {scale:.6g}")
        with rasterio.open(OUTPUT_RASTER,'r+') as src:
            for _,window in src.block_windows(1):
                data=src.read(1,window=window,boundless=True,fill_value=NODATA_VALUE)
                mask=(data!=NODATA_VALUE)&np.isfinite(data)
                if np.any(mask): data[mask]*=scale; src.write(data.astype('float32'),1,window=window)


# vector grid
log.info("Aggregating raster to vector grid (non-zero cells)...")
scale = GRID_CELL_SIZE / PIXEL_SIZE
agg_width = int(math.ceil(width/scale)); agg_height=int(math.ceil(height/scale))
agg_transform = from_origin(minx,maxy,GRID_CELL_SIZE,GRID_CELL_SIZE)
dst_agg = np.full((agg_height,agg_width),NODATA_VALUE,dtype='float32')
with rasterio.open(OUTPUT_RASTER) as src:
    reproject(source=rasterio.band(src,1),destination=dst_agg,
              src_transform=src.transform,src_crs=src.crs,
              dst_transform=agg_transform,dst_crs=src.crs,
              resampling=WarpResampling.average,
              src_nodata=NODATA_VALUE,dst_nodata=NODATA_VALUE)
mask_valid=(dst_agg!=NODATA_VALUE)&np.isfinite(dst_agg)&(dst_agg>0)
n_valid=int(np.count_nonzero(mask_valid))
log.info(f"Aggregated cells: {agg_width}x{agg_height} -> non-zero: {n_valid}")
if n_valid==0:
    vg=gpd.GeoDataFrame({'mean_intensity':[],'geometry':[]},crs=TARGET_CRS)
    vg.to_file(VECTOR_GRID,driver='GPKG')
else:
    results=[(geom,float(val)) for geom,val in rio_shapes(dst_agg,mask=mask_valid,transform=agg_transform)]
    geoms=[shapely_shape(g) for g,v in results]; vals=[v for g,v in results]
    vg=gpd.GeoDataFrame({'mean_intensity':vals,'geometry':geoms},crs=TARGET_CRS)
    vg.to_file(VECTOR_GRID,driver='GPKG')
    log.info(f"Saved {len(vg)} polygons to {VECTOR_GRID}")

# metadata & COG
meta_info={'created_at':datetime.datetime.utcnow().isoformat()+'Z',
           'source_vector':VECTOR_PATH,'lamp_count':len(lamps),'target_crs':TARGET_CRS,
           'pixel_size_m':PIXEL_SIZE,'gaussian_sigma_pixels':GAUSSIAN_SIGMA_PIX,
           'gaussian_sigma_meters':GAUSSIAN_SIGMA_PIX*PIXEL_SIZE,'chunk_size_pixels':CHUNK_SIZE,
           'grid_cell_size_m':GRID_CELL_SIZE,'normalize_total':bool(NORMALIZE_TOTAL),
           'nodata_value':float(NODATA_VALUE),'histogram_bins':int(NBINS_HIST),'seed':int(SEED)}
with open(METADATA_JSON,'w') as fh: json.dump(meta_info,fh,indent=2)
log.info(f"Metadata written to {METADATA_JSON}")

try:
    from rio_cogeo.cogeo import cog_translate
    from rio_cogeo.profiles import cog_profiles
    profile=cog_profiles.get('deflate')
    cog_translate(OUTPUT_RASTER,OUTPUT_COG,profile)
    log.info(f"COG created: {OUTPUT_COG}")
except Exception as e:
    log.warning(f"COG creation skipped/failed: {e}")

# verification & debuging
burn_check = 0
with rasterio.open(OUTPUT_RASTER) as src:
    for row_start in range(0, height, CHUNK_SIZE):
        for col_start in range(0, width, CHUNK_SIZE):
            r0=row_start; c0=col_start
            r1=min(height,row_start+CHUNK_SIZE)
            c1=min(width,col_start+CHUNK_SIZE)
            x0=minx+c0*PIXEL_SIZE
            y1=maxy-r0*PIXEL_SIZE
            idx = list(spatial_index.intersection((x0, maxy-r1*PIXEL_SIZE, minx+c1*PIXEL_SIZE, y1)))
            lamps_tile = lamps.iloc[idx]
            if len(lamps_tile)>0:
                burned = rasterize_additive(((g,1) for g in lamps_tile.geometry),
                                            out_shape=(r1-r0, c1-c0),
                                            transform_local=from_origin(x0,y1,PIXEL_SIZE,PIXEL_SIZE),
                                            dtype='uint32')
                burn_check += int(burned.sum())
log.info(f"Burn test: rasterized count sum = {burn_check}, original lamp count = {len(lamps)}")

def synthetic_additive_test():
    sminx=minx; smaxy=maxy
    synth = gpd.GeoDataFrame({'geometry':[Point(sminx+10,miny+10),Point(sminx+12,miny+10),Point(sminx+11,miny+12)]},crs=TARGET_CRS)
    out = rasterize_additive(((g,1) for g in synth.geometry), out_shape=(50,50),
                              transform_local=from_origin(sminx,smaxy,PIXEL_SIZE,PIXEL_SIZE))
    return int(out.sum())==len(synth)
synth_ok = synthetic_additive_test()
log.info(f"Synthetic additive test passed: {synth_ok}")

print("\nPipeline finished")
print(f"Raster: {OUTPUT_RASTER}")
print(f"COG: {OUTPUT_COG}")
print(f"Vector grid: {VECTOR_GRID}")
print(f"Histogram: {HIST_CSV}, {HIST_IMAGE}, {HIST_LOG_IMAGE}")
print(f"Metadata: {METADATA_JSON}")


upload file


Saving street-lamp-OSM.gpkg to street-lamp-OSM (1).gpkg


rows: 100%|██████████| 6/6 [00:15<00:00,  2.52s/it]
/tmp/ipython-input-536001957.py:268: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  meta_info={'created_at':datetime.datetime.utcnow().isoformat()+'Z',
Reading input: lighting_model_highres_disk.tif

Adding overviews...
Updating dataset tags...
Writing output to: lighting_model_highres_disk_cog.tif



Pipeline finished
Raster: lighting_model_highres_disk.tif
COG: lighting_model_highres_disk_cog.tif
Vector grid: lighting_vector_grid_nonzero.gpkg
Histogram: lighting_histogram.csv, lighting_histogram.png, lighting_histogram_log.png
Metadata: lighting_metadata.json


In [2]:
# analyze non-zero raster area & suggest polygon
import rasterio
from rasterio.features import shapes
import geopandas as gpd
from shapely.geometry import shape, box

RASTER_FILE = "lighting_model_highres_disk.tif"
VECTOR_GRID = "lighting_vector_grid_nonzero.gpkg"
OUTPUT_POLYGON = "nonzero_raster_area.gpkg"

# load raster and compute non-zero bounds
with rasterio.open(RASTER_FILE) as src:
    data = src.read(1, masked=True)  # masked array ignores nodata automatically
    mask = data > 0
    if mask.any():
        rows, cols = mask.nonzero()
        min_row, max_row = rows.min(), rows.max()
        min_col, max_col = cols.min(), cols.max()
        minx, maxy = src.xy(min_row, min_col)
        maxx, miny = src.xy(max_row, max_col)
        print(f"Non-zero raster bounds: {minx:.1f},{miny:.1f} to {maxx:.1f},{maxy:.1f}")
        suggested_poly = box(minx, miny, maxx, maxy)
    else:
        raise SystemExit("Raster has no non-zero values.")


Non-zero raster bounds: 466973.9,5919957.0 to 588248.9,5974427.0
